#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [4]:
docs

[Document(page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…', metadata={'source': 'speech.txt'}),
 Document(page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high sp

In [4]:
embeddings=OllamaEmbeddings(model="gemma:2b")
db=FAISS.from_documents(docs,embeddings)
db

In [5]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [6]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [7]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='4143073e-7288-43a4-8e60-30731dbdc4eb', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  np.float32(2946.962)),
 (Document(id='2551d44c-7f02-4dd7-b465-297c1e5e626a', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everyt

In [8]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.3048570156097412,
 -2.1358208656311035,
 0.2617272138595581,
 0.9925695061683655,
 0.6720275282859802,
 0.5375428795814514,
 -0.8922985792160034,
 0.14992740750312805,
 -0.10723201930522919,
 -0.7782111167907715,
 1.0805569887161255,
 -0.044918958097696304,
 -1.1135114431381226,
 1.0926815271377563,
 0.055055420845746994,
 -1.000118374824524,
 3.093967914581299,
 1.7301167249679565,
 1.1501288414001465,
 0.789290189743042,
 0.31042829155921936,
 -0.27346840500831604,
 0.33717668056488037,
 1.376485824584961,
 0.20864644646644592,
 -0.39612823724746704,
 -1.3367036581039429,
 -1.257700800895691,
 -0.5560897588729858,
 -2.107651948928833,
 -0.26638099551200867,
 -1.4359880685806274,
 1.2056469917297363,
 -0.9245194792747498,
 -0.4182955026626587,
 -0.2568396031856537,
 1.8546772003173828,
 0.5561342835426331,
 0.1531960517168045,
 -0.5393620133399963,
 0.34913739562034607,
 0.541814386844635,
 0.9417709112167358,
 -1.303439974784851,
 -1.3688313961029053,
 0.6399134993553162,
 0.06245

In [9]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='4143073e-7288-43a4-8e60-30731dbdc4eb', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='2551d44c-7f02-4dd7-b465-297c1e5e626a', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the

In [10]:
### Saving And Loading
db.save_local("faiss_index")

In [11]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [12]:
docs

[Document(id='4143073e-7288-43a4-8e60-30731dbdc4eb', metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(id='2551d44c-7f02-4dd7-b465-297c1e5e626a', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the